### zigbang 원룸 매물 데이터 수집 
- 조금 복잡한 형태의 크롤링 
- 결과 데이터를 수집하기 위해서 requests > response를 여러번 해야함 
- 웹사이트 분석을 잘해야함 
- 동이름 입력 > 매물정보 DF 

In [ ]:
# 1. 서비스 분석: url 
# (1) 동이름 입력 > 동이름에 해당하는 데이터 
# https://apis.zigbang.com/v2/search?leaseYn=N&q=동작구&serviceType=원룸
# (2) geohash code > 매물 아이디 
# https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang
# &geohash=wydme&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸
# (3) 매물 아이디 > 매물 정보 
# POST: https://apis.zigbang.com/v2/items/list
# (젤아래 view source) PARAMS: {"domain":"zigbang","withCoalition":true,"item_ids":[27091158,27127450,27013159]}

In [2]:
#!pip install geohash

Using legacy 'setup.py install' for geohash, since package 'wheel' is not installed.
    Running setup.py install for geohash ... done
You should consider upgrading via the '/home/ubuntu/.pyenv/versions/3.8.5/envs/python3/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
import geohash2
import requests 
import pandas as pd

In [2]:
# 1. 동이름 > 위도 경도 

In [4]:
addr = "성수동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
response

<Response [200]>

In [5]:
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.541725158691406, 127.04335021972656)

In [6]:
# 2. 위도, 경도 > geohash code

In [7]:
# precision이 커질수록 영역이 작아진다. 4정도 되면 서울 전지역 
geohash_code = geohash2.encode(lat, lng, precision=5)
geohash_code

'wydmd'

In [8]:
# 3. geohash code > 매물 아이디

In [9]:
geohash_code 
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash_code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [10]:
datas = response.json()["items"]
len(datas), datas[0]
item_ids = [data["item_id"] for data in datas] # list comprehension
len(item_ids), item_ids[:3]

(944, [27358761, 26934216, 26961954])

In [11]:
# 4. 매물 아이디 > 매물 정보 

In [20]:
# 매물 데이터는 최대 1000개씩 가져옴 
url = "https://apis.zigbang.com/v2/items/list"
params = {"domain":"zigbang","withCoalition":"true","item_ids":item_ids}
response = requests.post(url, params)
response

<Response [200]>

In [22]:
response.text

'{"items":[{"section_type":null,"item_id":27358761,"images_thumbnail":"https://ic.zigbang.com/ic/items/27358761/1.jpg","sales_type":"전세","sales_title":"전세","deposit":10000,"rent":0,"size_m2":62.81,"공급면적":{"m2":62.81,"p":"19"},"전용면적":{"m2":59.5,"p":"18"},"계약면적":null,"room_type_title":null,"floor":"1","floor_string":"1","building_floor":"2","title":"용산구 보광동 전세(1.5층.방3)1억원","is_first_movein":null,"room_type":"05","address":"용산구 보광동","random_location":{"lat":37.53041348648602,"lng":127.00259038439248},"is_zzim":false,"status":true,"service_type":"빌라","tags":[],"address1":"서울시 용산구 보광동","address2":null,"address3":null,"manage_cost":"0","reg_date":"2021-05-22T14:44:20+09:00","is_new":false},{"section_type":null,"item_id":26934216,"images_thumbnail":"https://ic.zigbang.com/ic/items/26934216/1.jpg","sales_type":"전세","sales_title":"전세","deposit":40000,"rent":0,"size_m2":36.32,"공급면적":{"m2":36.32,"p":"11"},"전용면적":{"m2":26.7,"p":"8.1"},"계약면적":null,"room_type_title":null,"floor":"6","floor_string":"

In [23]:
datas = response.json()["items"] #json은 딕셔너리로 파싱
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor", 
           "title", "manage_cost", "address1", "random_location"]
len(datas), datas[0]
df = pd.DataFrame(datas)[columns]
df.tail()

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1,random_location
939,27256123,월세,1000,60,26.4500,중,20,신설동 대로변 컨디션 좋은 오피스텔 임대,7.5,서울시 종로구 숭인동,"{'lat': 37.57347483172545, 'lng': 127.02321163..."
940,27300693,월세,1000,60,39.6694,고,20,🔻🔻🔻신설동역세권의 깨끗한 오피스텔,7,서울시 종로구 숭인동,"{'lat': 37.573476703712345, 'lng': 127.0225893..."
941,27306425,월세,1000,60,29.7500,중,20,💜전망좋은 오피스텔💜,7.5,서울시 종로구 숭인동,"{'lat': 37.57295414295621, 'lng': 127.02321427..."
942,27363369,월세,1000,60,40.0000,고,20,신설동역대로변 회사관리 안전하고 관리잘되는 오피스텔,7.5,서울시 종로구 숭인동,"{'lat': 37.57295175859404, 'lng': 127.02321370..."
943,27369942,월세,1000,60,29.7500,중,20,♥럭셔리 오피스텔♥,7.5,서울시 종로구 숭인동,"{'lat': 37.57347279843591, 'lng': 127.02321125..."


In [24]:
# precision이 커서 중동이 나오넹?
df = df[df["address1"].str.contains("성수")]
df.tail()

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1,random_location
39,27277834,월세,1000,45,19.83,4,4,초역세권 저렴한 원룸,5,서울시 성동구 성수동1가,"{'lat': 37.54714289566264, 'lng': 127.04529144..."
40,27208880,전세,24000,0,29.55,4,8,신축급 풀옵션 분리형 원룸,20,서울시 성동구 성수동1가,"{'lat': 37.548044792959054, 'lng': 127.0433300..."
43,27078515,월세,1000,45,23.14,2,4,♠조용한 주택가 방1+거실주방♠,0,서울시 성동구 성수동1가,"{'lat': 37.5503883604105, 'lng': 127.045466968..."
44,27293809,월세,1000,40,23.14,1,3,"분리형원룸, 즉시입주",0,서울시 성동구 성수동1가,"{'lat': 37.55087935181552, 'lng': 127.04483423..."
46,27089741,월세,1000,45,16.53,3,5,깨끗한 오픈형원룸,5,서울시 성동구 성수동1가,"{'lat': 37.55092579895423, 'lng': 127.04575505..."
